<a href="https://colab.research.google.com/github/fatemehabedin2/AIG100/blob/main/Lab_1_Basics_of_TensorFlow_and_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 3 Lab - Basics of TensorFlow and PyTorch

## Verifying GPU Availability

### TensorFlow GPU Check

In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print("GPU is available for TensorFlow!")
else:
    print("No GPU found for TensorFlow.")


TensorFlow version: 2.18.0
GPU is available for TensorFlow!


### PyTorch GPU Check

In [ ]:
import torch

print("PyTorch version:", torch.__version__)
if torch.cuda.is_available():
    print("GPU is available for PyTorch!")
else:
    print("No GPU found for PyTorch.")


PyTorch version: 2.6.0+cu124
GPU is available for PyTorch!


## Creating and Training a Simple Neural Network

### TensorFlow Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0     #levels of gray in each pixel  ... normalize the pixel value (0-1). 255 is the maximum pixel value in 8-bit grayscale images.
x_test = x_test / 255.0
y_train = to_categorical(y_train)     # Converts the labels (which are integers 0 to 9) into one-hot encoded vectors.
y_test = to_categorical(y_test)

# Define the model
model = Sequential([            # initializes a sequential model, which means layers are stacked one after the other in a linear fashion.
    Flatten(input_shape=(28, 28)),   # change the 2D to 1D vector. Converts each 28×28 2D image into a 1D vector of length 784.This is necessary because dense (fully connected) layers expect a 1D input.
    Dense(128, activation='relu'),   # Dense, A fully connected layer with 128 neurons. relu (Rectified Linear Unit) is used as the activation function:It introduces non-linearity. Formula: relu(x) = max(0, x). Learns 128 different features from the input.
    Dense(10, activation='softmax')   # Output layer with 10 neurons (one for each digit class: 0–9). Softmax converts raw scores into probabilities that sum to 1. The neuron with the highest value represents the model’s predicted digit.
])

# Compile the model
model.compile(          # configures the model for training.
    optimizer='adam',           # how the model updates its weights. Adam = Adaptive Moment Estimation. A very popular and efficient optimizer that combines the benefits of:Momentum (smooths updates using past gradients), RMSProp (scales learning rate based on past squared gradients). Usually works well out-of-the-box for many deep learning tasks.
    loss='categorical_crossentropy',  # is used for multi-class classification with one-hot encoded labels. It measures how far the predicted probabilities are from the true labels.
    metrics=['accuracy'])   # tracks the percentage of correct predictions

# Train the model
with tf.device('/GPU:0'):   # a TensorFlow device context manager that tells TensorFlow to run the operations inside the block on GPU #0 (the first available GPU).
    model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))   # recommended to use validation_data=(x_val, y_val) to prevent data leakage

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)   # evaluates the model on the entire test set once, after all training epochs are complete. It feeds all test images through the final trained version of your model (after epoch 5). It then calculates loss, accuracy
print(f'Test accuracy: {accuracy:.4f}')


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8799 - loss: 0.4262 - val_accuracy: 0.9543 - val_loss: 0.1476
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9635 - loss: 0.1209 - val_accuracy: 0.9707 - val_loss: 0.1018
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9778 - loss: 0.0748 - val_accuracy: 0.9719 - val_loss: 0.0920
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9828 - loss: 0.0547 - val_accuracy: 0.9755 - val_loss: 0.0778
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9867 - loss: 0.0432 - val_accuracy: 0.9774 - val_loss: 0.0761
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9751 - loss: 0.0894
Test accuracy: 0.9774


### PyTorch Implementation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader



# Load and preprocess the MNIST dataset
transform = transforms.Compose([               # creates a pipeline of image transformations that will be applied to each image in the dataset. The two transformations:
    transforms.ToTensor(),                     # Converts a PIL image or NumPy array to a PyTorch tensor. Also scales pixel values from [0, 255] to [0.0, 1.0]
    transforms.Normalize((0.5,), (0.5,))])     # Normalizes the tensor using:(x - 0.5) / 0.5 → scales [0, 1] to [-1, 1]. (0.5,), (0.5,): The first tuple is the mean, and the second is the standard deviation.
                                               # for Color images (RGB): Use per-channel normalization → (meanR, meanG, meanB), (stdR, stdG, stdB)

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)  # root='./data' → save the data to the ./data directory. train=True → load the training data. download=True → download it if not already present
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)              # train=False → means load the 10,000 test samples. the train parameter is a boolean flag that tells the loader: True → load training set. False → load test set

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)             # DataLoader: Efficient batch loading, Supports multi-threaded loading (using multiple CPU threads in parallel), Shuffling, sharding, and more. During training, the model updates its weights once per batch.
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)              # shuffle=True (only for training); Randomly shuffles the dataset at the beginning of each epoch. For testing, you typically don't shuffle. you want to evaluate it in a fixed, repeatable order.



# Define the model
class SimpleNN(nn.Module):                 # creates a new class SimpleNN that inherits from PyTorch’s nn.Module, the base class for all neural network models in PyTorch.
    def __init__(self):                    # __init__ is the constructor method of the class. It initializes the layers of your network. This method gets called automatically when you create a new object from a class (like model = SimpleNN()).
        super(SimpleNN, self).__init__()   # super() gives us access to the parent class ("superclass"). in this case, nn.Module, which SimpleNN inherits from.
                                           # super().__init__(), Tells Python to call the __init__() method of the parent class nn.Module, so it can set up all the internal machinery needed for a PyTorch neural network.”
                                           # self is a reference to the current object (or instance) of a class. When you define a class and then create an object from it, self lets that object refer to itself so it can access its own variables and methods.
        self.flatten = nn.Flatten()        # This layer converts the 2D input images (28x28 pixels from MNIST) into 1D vectors of size 784 (28×28 = 784). It’s necessary because fully connected layers expect flat vectors.
        self.fc1 = nn.Linear(28*28, 128)   #input size(28*28), output (128, number of neurons). First fully connected layer ("dense layer"):
        self.fc2 = nn.Linear(128, 10)      # second fully connected layer (fc2) is the output layer. These 10 values are called logits, which represent unnormalized predictions


    def forward(self, x):              # the forward() method defines the actual path your input data takes through the layers of the network. This function takes in x, the input tensor. self is the object itself, referring to the current instance of the model.
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))    # This passes the input through the first fully connected (linear) layer (fc1). Then it applies ReLU activation, which adds non-linearity. ReLU means: "Keep positive values, set all negative values to 0."
        x = self.fc2(x)                # No activation here because the next step usually applies softmax or cross-entropy loss, which handles it internally.
        return x                       # Returns the output logits (raw scores) from the network.

model = SimpleNN()        # creates an instance of your neural network class, SimpleNN.




# Define loss and optimizer
criterion = nn.CrossEntropyLoss()    # loss function. doesn't need normalization. has sort of softmax built-in. It combines: Softmax, which converts raw scores (logits) from the output layer into probabilities. Negative Log Likelihood (NLL), which penalizes wrong predictions by comparing predicted probabilities to the correct class label.
optimizer = optim.Adam(model.parameters(), lr=0.001)    # optimizer that will update your model's weights during training, to minimize the loss.
                                                        # model.parameters(): gives the optimizer access to all the learnable weights and biases in the model so it knows what to update.
                                                        # lr=0.001: learning rate, which controls how big the updates to the weights are. A small learning rate like 0.001 means small, careful updates — good for stable learning.
                                                        # If your learning rate is too low, training is slow and may get stuck in local minima. If it's too high, the model may not converge (never settle) or even diverge (get worse over time).

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')           # train on GPU if available, otherwise   on CPU.
model.to(device)        # Moves your model’s parameters and computations to the selected device.

# Train the model
num_epochs = 5
for epoch in range(num_epochs):        # epoch loop
    model.train()    # Tells PyTorch you’re training, not evaluating. Enables features like dropout and batch normalization (if your model uses them).
    for images, labels in train_loader:          # batch loop
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)                # Passes the images through the model. outputs = model’s predictions (logits for each class).
        loss = criterion(outputs, labels)      # calculating loss

        # Backward and optimize
        optimizer.zero_grad()                  # Clears previous gradients from the last step (important!). Otherwise, PyTorch would accumulate gradients across steps.
        loss.backward()                        # Backpropagation: Computes gradients of the loss w.r.t. model’s weights.
        optimizer.step()                       # Updates the model’s weights using gradients and learning rate

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')


100%|██████████| 9.91M/9.91M [00:00<00:00, 18.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 489kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.59MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.96MB/s]


Epoch [1/5], Loss: 0.1171
Epoch [2/5], Loss: 0.1318
Epoch [3/5], Loss: 0.0741
Epoch [4/5], Loss: 0.0508
Epoch [5/5], Loss: 0.0933
Test Accuracy: 95.92%


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader



# Load and preprocess the MNIST dataset
transform = transforms.Compose([               # creates a pipeline of image transformations that will be applied to each image in the dataset. The two transformations:
    transforms.ToTensor(),                     # Converts a PIL image or NumPy array to a PyTorch tensor. Also scales pixel values from [0, 255] to [0.0, 1.0]
    transforms.Normalize((0.5,), (0.5,))])     # Normalizes the tensor using:(x - 0.5) / 0.5 → scales [0, 1] to [-1, 1]. (0.5,), (0.5,): The first tuple is the mean, and the second is the standard deviation.
                                               # for Color images (RGB): Use per-channel normalization → (meanR, meanG, meanB), (stdR, stdG, stdB)

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)  # root='./data' → save the data to the ./data directory. train=True → load the training data. download=True → download it if not already present
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)              # train=False → means load the 10,000 test samples. the train parameter is a boolean flag that tells the loader: True → load training set. False → load test set

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)             # DataLoader: Efficient batch loading, Supports multi-threaded loading (using multiple CPU threads in parallel), Shuffling, sharding, and more. During training, the model updates its weights once per batch.
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)              # shuffle=True (only for training); Randomly shuffles the dataset at the beginning of each epoch. For testing, you typically don't shuffle. you want to evaluate it in a fixed, repeatable order.



# Define the model
class SimpleNN(nn.Module):                 # creates a new class SimpleNN that inherits from PyTorch’s nn.Module, the base class for all neural network models in PyTorch.
    def __init__(self):                    # __init__ is the constructor method of the class. It initializes the layers of your network. This method gets called automatically when you create a new object from a class (like model = SimpleNN()).
        super(SimpleNN, self).__init__()   # super() gives us access to the parent class ("superclass"). in this case, nn.Module, which SimpleNN inherits from.
                                           # super().__init__(), Tells Python to call the __init__() method of the parent class nn.Module, so it can set up all the internal machinery needed for a PyTorch neural network.”
                                           # self is a reference to the current object (or instance) of a class. When you define a class and then create an object from it, self lets that object refer to itself so it can access its own variables and methods.
        self.flatten = nn.Flatten()        # This layer converts the 2D input images (28x28 pixels from MNIST) into 1D vectors of size 784 (28×28 = 784). It’s necessary because fully connected layers expect flat vectors.
        self.fc1 = nn.Linear(28*28, 128)   #input size(28*28), output (128, number of neurons). First fully connected layer ("dense layer"):
        self.fc2 = nn.Linear(128, 10)      # second fully connected layer (fc2) is the output layer. These 10 values are called logits, which represent unnormalized predictions


    def forward(self, x):              # the forward() method defines the actual path your input data takes through the layers of the network. This function takes in x, the input tensor. self is the object itself, referring to the current instance of the model.
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))    # This passes the input through the first fully connected (linear) layer (fc1). Then it applies ReLU activation, which adds non-linearity. ReLU means: "Keep positive values, set all negative values to 0."
        x = self.fc2(x)                # No activation here because the next step usually applies softmax or cross-entropy loss, which handles it internally.
        return x                       # Returns the output logits (raw scores) from the network.

model = SimpleNN()        # creates an instance of your neural network class, SimpleNN.




# Define loss and optimizer
criterion = nn.CrossEntropyLoss()    # loss function. doesn't need normalization. has sort of softmax built-in. It combines: Softmax, which converts raw scores (logits) from the output layer into probabilities. Negative Log Likelihood (NLL), which penalizes wrong predictions by comparing predicted probabilities to the correct class label.
optimizer = optim.Adam(model.parameters(), lr=0.001)    # optimizer that will update your model's weights during training, to minimize the loss.
                                                        # model.parameters(): gives the optimizer access to all the learnable weights and biases in the model so it knows what to update.
                                                        # lr=0.001: learning rate, which controls how big the updates to the weights are. A small learning rate like 0.001 means small, careful updates — good for stable learning.
                                                        # If your learning rate is too low, training is slow and may get stuck in local minima. If it's too high, the model may not converge (never settle) or even diverge (get worse over time).

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')           # train on GPU if available, otherwise   on CPU.
model.to(device)        # Moves your model’s parameters and computations to the selected device.

# Train the model
num_epochs = 5
for epoch in range(num_epochs):        # epoch loop
    model.train()    # Tells PyTorch you’re training, not evaluating. Enables features like dropout and batch normalization (if your model uses them).
    running_loss = 0.0
    for images, labels in train_loader:          # batch loop
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)                # Passes the images through the model. outputs = model’s predictions (logits for each class).
        loss = criterion(outputs, labels)      # calculating loss

        # Backward and optimize
        optimizer.zero_grad()                  # Clears previous gradients from the last step (important!). Otherwise, PyTorch would accumulate gradients across steps.
        loss.backward()                        # Backpropagation: Computes gradients of the loss w.r.t. model’s weights.
        optimizer.step()                       # Updates the model’s weights using gradients and learning rate
        running_loss += loss.item()            # .item(): prints a float

    avg_loss = running_loss / len(train_loader)               # calculating the average loss for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')

# Evaluate the model
model.eval()           # Puts the model in evaluation mode
with torch.no_grad():   # Temporarily turns off gradient tracking to save memory and speed up computations, since we’re not training here
    correct = 0
    total = 0
    for images, labels in test_loader:        # Goes through the test dataset batch by batch.
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)          # Gets the model’s raw predictions (logits) for each image.
        _, predicted = torch.max(outputs.data, 1)   # Selects the index of the highest score for each image. torch.max(tensor, dim) == torch.max(outputs.data, 1) finds the maximum value along a specific dimension (We're taking the maximum across columns (i.e., across classes) for each row/image.). Returns two things: 1-The maximum value (not needed here, so we assign it to _) 2- The index of that maximum → this is the predicted class.
        total += labels.size(0)                     # counting how many total predictions were made. size(0): how many labels are in the batch.
        correct += (predicted == labels).sum().item()   # predicted is tensor of class indices, one for each image in the batch. labels is a tensor containing the true class labels for a batch of images

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

100%|██████████| 9.91M/9.91M [00:11<00:00, 899kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 134kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.08MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.09MB/s]


Epoch [1/5], Average Loss: 0.3516
Epoch [2/5], Average Loss: 0.1786
Epoch [3/5], Average Loss: 0.1320
Epoch [4/5], Average Loss: 0.1093
Epoch [5/5], Average Loss: 0.0939
Test Accuracy: 96.80%
